# USC CODE START

In [26]:
import pandas as pd
import geopandas  as gpd
import os
import json
from sqlalchemy import engine
# from PIL import Image
# from PIL.ExifTags import TAGS
from datetime import datetime
from sqlalchemy import create_engine
import gdal2tiles  # not installed on USC computers
import xml.etree.ElementTree as ET
from shapely import wkt
from shapely.validation import make_valid
import glob
import subprocess
import tempfile
import time
import shutil

## FUNC COVERAGES

In [2]:
def humanbytes(B):
    """Return the given bytes as a human friendly KB, MB, GB, or TB string."""
    B = float(B)
    KB = float(1024)
    MB = float(KB ** 2) # 1,048,576
    GB = float(KB ** 3) # 1,073,741,824
    TB = float(KB ** 4) # 1,099,511,627,776

    if B < KB:
        return '{0} {1}'.format(B,'Bytes' if 0 == B > 1 else 'Byte')
    elif KB <= B < MB:
        return '{0:.2f} KB'.format(B / KB)
    elif MB <= B < GB:
        return '{0:.2f} MB'.format(B / MB)
    elif GB <= B < TB:
        return '{0:.2f} GB'.format(B / GB)
    elif TB <= B:
        return '{0:.2f} TB'.format(B / TB)

In [3]:
def mk_coverage(env, img_path, cov_path):
    # need to call/open command line which uses gdal cli functions to process coverages
    # change directiory
    os.chdir(env)
    # indexes are coverages i.e coverage creation from tiff to new folder
    cmd = f'gdaltindex {cov_path} {img_path}'
    subprocess.call(cmd,shell=True)
    # print(f'coverage created -- {cov_path}')
    return(cov_path)

In [4]:
def create_coverages(folder,env):
    # ------------------------------------------------- #
    dfdict = {'img_name':[], 'fips':[],'dt_year':[],'img_num':[], 'img_size':[],
            'meta_size':[],'meta_unit':[],'state':[],'meta_last_acc':[],'img_path':[]}
    # ------------------------------------------------- #
    dfdict_cov_geom = {'img_name':[], 'area_sqkm':[], 'shape':[]}
    # ------------------------------------------------- #
    for image in glob.glob(os.path.join(folder, '*.tif')):
        image = image.replace('\\','/')
        # ------------------------------------------------- #
        img_path    = os.path.join(folder, image)
        stats       = os.stat(img_path)
        # temp path for coverage geojson
        # create a temporary folder using library tempfile
        temp_dir = tempfile.mkdtemp()
        temp_cov_path = os.path.join(temp_dir, 'temp_coverage.geojson')
        # print(temp_cov_path)
        # ------------------------------------------------- #
        # get the image name without extension
        # img_name = image.split('\\')[-1].replace('.tif','') ; print(img_name)
        img_name   = os.path.basename(image).replace('.tif','') 
        imgsplit    = img_name.split('_')
        # ------------------------------------------------- #
        (fips, dt_year, img_num, img_size,
            meta_size, meta_lastmod, meta_lastacc) = (imgsplit[0], imgsplit[1],
                                                    imgsplit[3], imgsplit[5], humanbytes(round(stats.st_size)),
                                                        datetime.fromtimestamp(stats.st_mtime),
                                                        datetime.fromtimestamp(stats.st_atime))
        # ------------------------------------------------- #
        # make coverages image outlines vector version
        cov_path        = mk_coverage(env, img_path, temp_cov_path)
        crs_string      = 'EPSG:3857'
        # coverages are now a spatial file that needs tiff metadata added to it via appending a list
        gdf             = gpd.read_file(cov_path, crs=crs_string)
        # precautionary check for odd polys -- not needed 
        gdf             = gdf.dissolve()
        # double checking and making sure geometry is valid -- not need but good practice
        gdf['geometry'] = gdf['geometry'].apply(make_valid)
        gdf['area_sqkm']= gdf['geometry'].map(lambda p: p.area / 10**6)
        sqkm            = int(round(sum(gdf.area_sqkm),2))
        geom            = gdf['geometry'].values[0]
        # ------------------------------------------------- #
        dfdict['img_name'].append(str(img_name))      ;  dfdict['fips'].append(str(fips)),
        dfdict['dt_year'].append(str(dt_year))        ;  dfdict['img_num'].append(str(img_num)),
        dfdict['img_size'].append(str(img_size))      ;  dfdict['meta_size'].append(str(meta_size).replace(' MB',''))
        dfdict['meta_last_acc'].append(str(meta_lastacc))  ;  dfdict['img_path'].append(str(img_path))
        dfdict['meta_unit'].append('MB')                   ;  dfdict['state'].append('South Carolina')
        # ------------------------------------------------- #
        dfdict_cov_geom['img_name'].append(str(img_name))
        dfdict_cov_geom['area_sqkm'].append(sqkm)#(int(round(sum(gdf.area_sqkm),2)))
        dfdict_cov_geom['shape'].append(geom)
    # merge both dictionaries on img_name
    merge_dict = {**dfdict, **dfdict_cov_geom}
    return(pd.DataFrame(merge_dict))


In [5]:
# function to check if a file exists, if not create it
def ck_file(x):
    if not os.path.exists(x):os.makedirs(x)
    return(x)

In [20]:
def create_json(xml_path, minzoom, maxzoom):
    # read xml file
    tree = ET.parse(xml_path)
    root = tree.getroot()
    # get the name of the image
    name = root.find('Title').text
    # get the bounds of the image
    bounds = root.find('BoundingBox').attrib
    # get the minx, miny, maxx, maxy
    minx = bounds['minx']
    miny = bounds['miny']
    maxx = bounds['maxx']
    maxy = bounds['maxy']
    # create a dictionary
    json_dict = {'name':name, 'version':'1.0.0','description':'', 'type':'overlay',
                 'format':'png','minzoom':minzoom, 'maxzoom':maxzoom, 'bounds':f'{minx},{miny},{maxx},{maxy}',
                'scale':'1', 'profile':'mercator'}
    return(json_dict)

# Global Variables

In [6]:
# establish global vars for county, years, cov folder name etc..
county         = 'D:\\45085-Sumter' #G:\\45013-Beaufort'
yr_folder_list = glob.glob(f'{county}/*') # var contents ex : ['G:\\45013-Beaufort\\45013_TIFF_1939', 'G:\\45013-Beaufort\\45013_TIFF_1941']
env            = 'C:\\Users\\terra\\anaconda3\\envs\\pygdal' # need to call/open command line which uses gdal cli functions to process coverages, mosaics/merges and tiles

In [34]:
for year in yr_folder_list:
    # check if year is a folder
    if not os.path.isdir(year):continue
    # coverages folder path
    cov_path = ck_file(os.path.join(year,'coverage_by_image'))
    # call function to create coverages and store in dataframe
    df  = create_coverages(year,env)
    # make the df spatial
    gdf = gpd.GeoDataFrame(df, geometry=df['shape'], crs=3857)
    gdf = gdf.drop(columns=['shape'])
    # make a dissolved/unioned version
    gdf_new = gdf.dissolve(by='state')
    # store coverages in appropriate folder
    gdf.to_file(os.path.join(cov_path,'coverages.geojson'), driver="GeoJSON") 
    gdf_new.to_file(os.path.join(cov_path,'coverages_dissolved.geojson'), driver="GeoJSON") 

# 16 to 8 bit Merge/Mosaic

In [54]:
# Reference: https://www.youtube.com/watch?v=sBBMKbAj8XE
# using the second method because it create a vertual environment that won't bog down memory
# hopefully no need for chunking
# os.chdir(env)
for year in yr_folder_list:
    os.chdir(env)
    # ------------------------------------------------- #
    if not os.path.isdir(year):continue
    # create a merged geotiff folder
    merged_tiff     = ck_file(os.path.join(year,'merged_tiff'))
    merged_tiff_txt = os.path.join(merged_tiff,'merged_tiff_list.txt')
    merged_tiff_vrt = os.path.join(merged_tiff,'merged_tiff.vrt')
    merged_geotiff  = os.path.join(merged_tiff,'merged_geotiff.tif')
    # list all tiffs in yr folder and store as merged_tiff_list.txt
    # purpose is to use the list as refference when updating the merged geotif and not reprocess all tiffs
    # check if merged_tiff_list.txt exists, if not create it
    # --- #
    org_tiff_list = glob.glob(f"{year}/*.tif")
    # ------------------------------------------------- #
    if not os.path.exists(merged_tiff_txt):
        # list all tiffs in yr folder
        tiff_list = glob.glob(f"{year}/*.tif")
        # write tiff_list to txt file
        with open(merged_tiff_txt, 'w') as f:
            for item in tiff_list:
                f.write('%s\n' % item)
    # ------------------------------------------------- #
    else:
        # read in the txt file and store as tiff list to be used to update the merged geotif
        with open(merged_tiff_txt, 'r') as f:
            tiff_list = f.read().splitlines()
    # ------------------------------------------------- # 
    # UPDATE CLAUSE: compare the org_tiff_list to the tiff_list if there are any new tiffs, create var new_tiff_list
    s = set(tiff_list)
    new_tiff_list = [x for x in org_tiff_list if x not in s]
    # check if geotiff exists, if it does, rename it to merged_geotiff_old.tif and update new tiffs list to include it
    if os.path.exists(merged_geotiff):
        os.rename(merged_geotiff,merged_geotiff.replace('.tif','_old.tif'))
        if merged_geotiff.replace('.tif','_old.tif') not in new_tiff_list:
            new_tiff_list.append(merged_geotiff.replace('.tif','_old.tif'))
    # convert to string in order to use gdalbuildvrt
    tiff_string = ' '.join(new_tiff_list[:2])
    print(tiff_string)
    # --- #
    # gdalbuildvrt to create a vrt file from the tiff list
    cmdV = f'''gdalbuildvrt -srcnodata 0 -vrtnodata 0 -overwrite "{merged_tiff_vrt}" {tiff_string}'''
    # subprocess.call(cmdV,shell=True)
    time.sleep(1)
    print(f'vertual file {os.path.exists(merged_tiff_vrt)}')
    # merge the vertual file to a webp compressed geotiff in 8bit format
    cmdM = f'''gdal_translate -a_nodata 0.0 -ot Byte -of GTiff {merged_tiff_vrt.replace('"','')} {merged_geotiff.replace('"','')}'''
    # subprocess.call(cmdM,shell=True) # 3 files took 11mins to process; however updating  with 2 tiffs and 1 merged geotiff took 3mins
    time.sleep(2)
    print(f'merged file {os.path.exists(merged_geotiff)}')
    # ------------------------------------------------- #
    # update text file with new tiffs overwrite the old one
    with open(merged_tiff_txt, 'a') as f:
        tiffs = '\n'.join(new_tiff_list+'\n'+tiff_list)
        f.write(tiffs)
    # ------------------------------------------------- #
    # remove the old merged geotiff because it is no longer needed
    shutil.rmtree(merged_geotiff.replace('.tif','_old.tif'), ignore_errors=True)
    print('old merged geotiff removed')

D:\45085-Sumter\45085_TIFF_1980\45085_1980_179_0005_x_9.tif D:\45085-Sumter\45085_TIFF_1980\45085_1980_179_0007_x_9.tif
vertual file True
merged file True


# Tile & Make Json

In [ ]:
for year in yr_folder_list:
    tile_cov        = ck_file(os.path.join(year,'tiled_coverages'))
    merged_geotiff  = os.path.join(merged_tiff,'merged_geotiff.tif')
    json_file       = os.path.join(tile_cov,'tilemapresource.json')
    xml_file        = os.path.join(tile_cov,'tilemapresource.xml')
    # UPDATE CLAUSE : check if tilemapresource.json exists, if so empty the folder and reprocess
    if os.path.exists(json_file):
        shutil.rmtree(tile_cov)
        os.mkdir(tile_cov)
    # source: https://gdal2tiles.readthedocs.io/en/latest/readme.html
    gdal2tiles.generate_tiles(merged_geotiff, tile_cov, zoom='2-5', processes=4,
                              s_srs='EPSG:3857', profile='mercator', resampling='average',
                              webp=True)
    # make a json file for the tileset
    j = create_json(xml_file, 2, 5)
    with open(json_file, 'w') as outfile:
        json.dump(j, outfile)
        
        

# Junk Code

In [ ]:
    # f'''gdal2tiles.bat -p mercator -z 8-10 -w all -r average -a 0.0
    #         "{out_merge}" "{tile_path}"'''
    # begin merging tiffs process
    # build vertual raster that a mosaic of all tiffs in tiff_list
    # gdal.BuildVRT( dst file: xml file that stores the list of tiffs,
    #                src file: list of tiffs,
    #                separate: True means each tiff is a band, False means all tiffs are one band,
    #                 all tiffs have multiple bands so separate=True)
    # vrt = gdal.BuildVRT(merged_tiff_vrt, tiff_list)
    # print('vertual raster created')
    
    # gdal.Translate( dst file: merged geotiff,
    #                src file: vrt,
    #             format: geotiff,
    #             compress 16 bit to 8 bit
    # gdal.Translate(merged_geotiff, vrt, format='GTiff')
    # print('geotiff created')
    # close vrt
    # vrt = None
    # print('vrt closed')

In [ ]:

# def tiff_to_tile(folder):
#     bit_webp_ls     = []
#     need_process    = []
#     txt_ls          = []
#     # -------------------- #
#     # create a temp directory to store 8bit.webp files
#     temp_dir = tempfile.mkdtemp()
#     # -------------------- #
#     # list of tiff files
#     tiff_ls = [t.replace('\\','/') for t in glob.glob(os.path.join(folder, '*.tif'))]
#     tiff_ls = [t.split('/')[-1].replace('.tif','') for t in tiff_ls]
#     # -------------------- #
    
#     # read list from text file
#     with open(os.path.join(folder,'cov_8bit/bit_webp_ls.txt'), 'r') as f:
#         for line in f:
#             txt_ls.append(line.split(',')) 
#     # flatten list
#     txt_ls = [item for sublist in txt_ls for item in sublist]  
#     # print(f'list from text file: {txt_ls}')
#     # print(f'list of tiff files: {tiff_ls}')
    
#     # -------------------- #
    
#     # if tiff names are not in txt file, add to need_process list
#     if txt_ls not in tiff_ls:
#         # append list of tiff files that are not in txt file
#         need_process.append([x for x in tiff_ls if x not in txt_ls])
#     # flatten list
#     need_process = [item for sublist in need_process for item in sublist]
#     # print(f'list of tiff files that need to be processing: {need_process}')
    
#     # -------------------- #
    
#     for image in need_process: 
#         # folders and files var establishment
#         img_name = image.split('/')[-1].replace('.tif','')
#         org_tif  = os.path.join(folder, f'{img_name}.tif')
#         bit_webp   = os.path.join(temp_dir, f'{img_name}_8bit.webp')
#         # -------------------- #
#         # print(f'processing {img_name}...')
#         # convert tiff to 8bit webp format
#         cmd_8bit = f'gdal_translate -ot Byte -scale {org_tif} {bit_webp}'
#         # converte tiff to 8bit webp format
#         subprocess.call(cmd_8bit,shell=True)
#         # append to list of 8bit.webp files
#         bit_webp_ls.append(img_name)  
#         # print(f'{img_name} completed')
    
#     # -------------------- #
    
#     # overwrite current bit_webp_ls.txt & save list of 8bit.webp files to a text file for future use
#     with open(os.path.join(folder,'cov_8bit/bit_webp_ls.txt'), 'w') as f:
#         # write each item from the list to text file
#         for item in bit_webp_ls:
#             f.write("%s" % item)
#     print(f'list of 8bit.webp files: {bit_webp_ls}') 
#     ## SOLVE FOR REFRESHING THE LIST OF 8BIT.WEBP FILES ##
    
#     ## FINAL VERSIION NEEDS VARS FOR 3 IN-YEAR FOLDERS ##
    
#     # -------------------- #
#     # mosaic all 8bit.webp files in temp_dir
    
#     # list of all 8bit.webp files in temp_dir
#     webp_files = glob.glob(os.path.join(temp_dir, '*.webp'))
#     tile_folder = os.path.join(folder,'mosaic').replace('\\','/')
#     # gdal mosaic all files in temp folder
#     cmd_mosaic = f'''gdalbuildvrt -overwrite -resolution average
#      -r nearest -input_file_list {webp_files} {tile_folder}/mosaic.vrt'''
#     subprocess.call(cmd_mosaic,shell=True) 
#     print(f'mosaic.vrt created\n{tile_folder}')
#     # print(webp_files)
    
#     ## TROUBLESHOOT WHY gdalbuildvrt IS NOT WORKING ##
    
    
    
    
# tiff_to_tile(folder)

#### merge 'nodata' not set BYTE (8BIT) output 0 | 

In [ ]:
'''gdal_merge.bat -a_nodata 0.0 -ot Byte -of GTiff -o
         "C:/Users/ArcGIS Student/AppData/Local/Temp/processing_zlStPW/5b9bddc4b2ae459d8f60525152222319/OUTPUT.tif" 
--optfile "{out}"
'''

In [ ]:
# # get list of files files.. 
# file_list = glob.glob("c:\data\....\*.tif")

# files_string = " ".join(file_list)
# # code needed to batch list based on large number of files, i.e list greater than 100 batch by increase by 1 for every 50 then dump temp file memory.
# out_merge    = # merge_image_folder\\merge*date YY-MM-dd

# merge_cmd = f'''gdal_merge.bat -a_nodata 0.0 -ot Byte -of GTiff 
#             -o         {files_string}
#             --optfile "{out_merge}"
#             '''


# # once merge is completed, tile merge in tile folder
# tile_path = # tile_folder\\tiled_*date YY-MM-dd


# # tile command
# tile_cmd = f'''gdal2tiles.bat -p mercator -z 8-10 -w all -r average -a 0.0
#             "{out_merge}" "{tile_path}"'''


In [ ]:
'''gdal2tiles.bat -p mercator -z 8-10 -w all -r average -a 0.0 "C:/Users/ArcGIS Student/AppData/Local/Temp/processing_zlStPW/f5bca6e8f38c4881b3ad500030680668/OUTPUT.tif" "C:/Users/ArcGIS Student/AppData/Local/Temp/processing_zlStPW/2bfcee2fbc4a41f996931ed107d0ee7e/OUTPUT"'''

## 16bit to 8bit Merge

In [ ]:
# for yr_path in yr_folder_list:
# 	# build recursive list of tiffs in the folder
# 	my_list = glob.glob(f'{yr_path}\\*.tif')
# 	# chunck list into manageable bites 
# 	chunk_size = 35 # 35 was chosen at random and will later be based on folder size and processing will later be parallelized as my previous version was.
# 	while my_list:
# 		chunk, my_list = my_list[:chunk_size], my_list[chunk_size:]
# 		for num, path in enumerate(chunk):
# 		# temp path and stuff here
# 			print(num, path)
# 		# outside of loop... this is where the mk_mosaic will live
# 		print('new')


In [ ]:
# # function to merge/mosaic files -- NOTE:: check algorthms behind both to ensure merging is the best method
# tif_ls = ['G:\\45013-Beaufort\\45013_TIFF_1979\\45013_1979_0178_135L_x_24.tif','G:\\45013-Beaufort\\45013_TIFF_1979\\45013_1979_0178_135R_x_24.tif', 'G:\\45013-Beaufort\\45013_TIFF_1979\\45013_1979_0178_136R_x_24.tif']
# mosaic_path = 'F:\\45085-Sumter'
# # def mk_mosaic(env, tif_ls, mosaic_path):
#     # need to call/open command line which uses gdal cli functions to process coverages
#     # change directiory
# os.chdir(env)
# # indexes are coverages i.e coverage creation from tiff to new folder
# cmd = f'gdal_merge.py -a_nodata 0.0 -ot Byte -of GTiff -o {mosaic_path} --optfile {tif_ls}'   # f'gdaltindex {cov_path} {img_path}'
# subprocess.call(cmd,shell=True)
#     # print(f'coverage created -- {cov_path}')
#     # return(cov_path)

In [ ]:
# gdal_merge.py -n 0 -v -o mosaic_31.tif --optfile tiff_list.txt

# gdal_merge.bat -a_nodata 0.0 -ot Byte -of GTiff 
#             -o         {files_string}
#             --optfile "{out_merge}"

In [ ]:

# env     = 'C:/Users/ArcGIS Student/anaconda3/envs/geospatial_23'#'D:/County_fips/fips_year'